In [0]:
%run "/Users/hilla.abramov@gmail.com/utils/s3_connector"

In [0]:
%run "/Users/hilla.abramov@gmail.com/utils/topic_batch_processor"

In [0]:
%run "/Users/hilla.abramov@gmail.com/utils/config_dicts"

In [0]:
%run "/Users/hilla.abramov@gmail.com/cleaning_methods/pinterest_post_cleaning"

In [0]:
%run "/Users/hilla.abramov@gmail.com/cleaning_methods/geolocation_cleaning"

In [0]:
%run "/Users/hilla.abramov@gmail.com/cleaning_methods/users_cleaning"

## Extracting the data

In [0]:
# initiate instance of the S3Connector class (defined in s3_connector notebook in utils folder), using the s3_config_dict (defined in the config_dicts notebook also in the utils folder)
s3_connection = S3Connector(s3_config_dict)
s3_connection.mount_S3_bucket()

# initiate instances of the TopicBatchProcessor class (defined in topic_batch_processor notebook in utils folder), per topic, desired topic names can be configured in the config_dicts notebook in the utils folder
geo_data_processor = TopicBatchProcessor(s3_connection, topic_names_dict["geolocation"])
pin_data_processor = TopicBatchProcessor(s3_connection, topic_names_dict["pinterest_posts"])
user_data_processor = TopicBatchProcessor(s3_connection, topic_names_dict["users"])

Your S3 bucket has already been mounted under the name of /mnt/user_0a0223c10829_s3_bucket

In [0]:
# read JSON files from S3 bucket into dataframe for each topic, which is saved as an attribute of the respective TopicBatchProcessor instance
for topic_processor in [geo_data_processor, pin_data_processor, user_data_processor]:
    topic_processor.read_json_files_from_s3_bucket_to_df()

## Data Cleaning

In [0]:
# bespoke cleaning methods imported from notebooks in cleaning_methods folders
pin_data_processor.clean_dataframe(clean_pin_df)
geo_data_processor.clean_dataframe(clean_geo_df)
user_data_processor.clean_dataframe(clean_user_df)

## Data Queries

In [0]:
# to enable sql queries through spark, creating temporary views of each dataframe in the spark session
import re

for topic_processor in [geo_data_processor, pin_data_processor, user_data_processor]:
    # creates table names "hilla_geo", "hilla_pin" and "hilla_user" to avoid interference with other session users
    sql_table_name = re.sub(r'^\w+\.', 'hilla_', topic_processor._topic_name)
    topic_processor._cleaned_batch_data.createOrReplaceTempView(sql_table_name)

In [0]:
%run "/Users/hilla.abramov@gmail.com/data_queries"

+--------------------+--------------+--------------+
 country| category|category_count|
+--------------------+--------------+--------------+
 Afghanistan| education| 9|
 Albania| art| 14|
 Algeria| quotes| 19|
 American Samoa| tattoos| 7|
 Andorra| tattoos| 7|
 Angola| education| 4|
 Anguilla|diy-and-crafts| 4|
Antarctica (the t...| tattoos| 4|
 Antigua and Barbuda| christmas| 3|
 Antigua and Barbuda| art| 3|
 Argentina| tattoos| 9|
 Armenia| art| 4|
 Aruba| art| 7|
 Australia| christmas| 2|
 Australia| mens-fashion| 2|
 Australia| education| 2|
 Austria| travel| 3|
 Austria| tattoos| 3|
 Azerbaijan| finance| 4|
 Bahamas|diy-and-crafts| 3|
+--------------------+--------------+--------------+
only showing top 20 rows

+---------+--------------+--------------+
post_year| category|category_count|
+---------+--------------+--------------+
 2018| art| 18|
 2018| beauty| 15|
 2018| christmas| 23|
 2018|diy-and-crafts| 19|
 2018| education| 17|
 2018|event-planning| 11|
 2018| finance| 10|
 2018| home-decor| 6|
 2018| mens-fashion| 12|
 2018| quotes| 18|
 2018| tattoos| 13|
 2018| travel| 20|
 2018| vehicles| 13|
 2019| art| 17|
 2019| beauty| 8|
 2019| christmas| 15|
 2019|diy-and-crafts| 16|
 2019| education| 16|
 2019|event-planning| 10|
 2019| finance| 7|
+---------+--------------+--------------+
only showing top 20 rows

+--------------------+--------------------+--------------+
 country| poster_name|follower_count|
+--------------------+--------------------+--------------+
 Afghanistan| 9GAG| 3000000|
 Albania| The Minds Journal| 5000000|
 Algeria| YourTango| 942000|
 American Samoa| Mamas Uncut| 8000000|
 Andorra|Teachers Pay Teac...| 1000000|
 Angola| Tastemade| 8000000|
 Anguilla|Kristen | Lifesty...| 92000|
Antarctica (the t...| Refinery29| 1000000|
 Antigua and Barbuda|Country Living Ma...| 1000000|
 Argentina| Cheezburger| 2000000|
 Armenia|Michelle {CraftyM...| 892000|
 Aruba| GQ Magazine| 874000|
 Australia| Mujer de 10 | 2000000|
 Austria| Diply| 251000|
 Azerbaijan| Style Me Pretty| 6000000|
 Bahamas| Handmade Charlotte| 948000|
 Bahrain|Alea Milham | Pre...| 332000|
 Bangladesh|Better Homes and ...| 4000000|
 Barbados|The Creativity Ex...| 410000|
 Belarus| Something Turquoise| 312000|
+--------------------+--------------------+--------------+
only showing top 20 rows

+-------+--------------+
country|follower_count|
+-------+--------------+
 Angola| 8000000|
+-------+--------------+

+---------+------------+--------------+
age_group| category|category_count|
+---------+------------+--------------+
 50+|mens-fashion| 10|
 18-24| tattoos| 55|
 36-50| quotes| 20|
 25-35| christmas| 26|
+---------+------------+--------------+

+---------+---------------------+
age_group|median_follower_count|
+---------+---------------------+
 50+| 1000|
 18-24| 84000|
 36-50| 6000|
 25-35| 21000|
+---------+---------------------+

+---------+-------------------+
post_year|number_users_joined|
+---------+-------------------+
 2015| 379|
 2016| 424|
 2017| 150|
+---------+-------------------+

+---------+---------------------+
post_year|median_follower_count|
+---------+---------------------+
 2015| 123000|
 2016| 19000|
 2017| 2000|
+---------+---------------------+

+---------+---------+---------------------+
age_group|post_year|median_follower_count|
+---------+---------+---------------------+
 18-24| 2015| 211000|
 25-35| 2015| 44000|
 36-50| 2015| 10000|
 50+| 2015| 1000|
 18-24| 2016| 40000|
 25-35| 2016| 19000|
 36-50| 2016| 8000|
 50+| 2016| 1000|
 18-24| 2017| 10000|
 25-35| 2017| 1000|
 36-50| 2017| 3000|
 50+| 2017| 1000|
+---------+---------+---------------------+